# Markov Model class definition

This notebook defines a constructor for a Markov model class.

In [19]:
# Put all the imports at the beginning

import numpy as np

Although it's not the only way of defining a Markov model, for the moment, I'm going to do the definition by taking arguments in the constructor that represent a distribution of transitions.

In [44]:
class MarkovModel:
    
    def __init__(self, transitions_ls, initialStates_ls=[], extraStates_ls=[]):
        '''
        Take a list of initial states, and a list of pairs of transitions
        between states. Create a Markov model based on the distribution of
        initial states, and distribution of transitions.
        
        extraStates_ls is a list of additional states which do not appear
        in the two lists initialStates_ls and transitions_ls.
        
        If initialStates_ls is empty, assume an equal distribution over all
        the states obtained from the transitions and the extra states.
        '''
        
        # First, build the list of all states in the model
        self.stateIndex_ls=list({x for x in initialStates_ls}. \
                                 union({x for (x, y) in transitions_ls}). \
                                 union({y for (x, y) in transitions_ls}). \
                                 union(set(extraStates_ls)))
        self.stateIndex_ls.sort() # just for aesthetics

        # Now build an array that contains the initial states
        if initialStates_ls==[]:
            initialStates_ls=self.stateIndex_ls        
        self.initialState_ar=np.array([initialStates_ls.count(state) 
                                       for state in self.stateIndex_ls])
        # and normalise the values so the prob.s sum to 1
        self.initialState_ar=self.initialState_ar/np.sum(self.initialState_ar)
        
        # Now build an array that encodes the transitions
        self.transitionMatrix_ar=np.zeros((len(self.stateIndex_ls), 
                                           len(self.stateIndex_ls)), 
                                          dtype=np.float)  # Normally int, but we're
                                                           # going to normalise
        for (x, y) in transitions_ls:
            self.transitionMatrix_ar[self.stateIndex_ls.index(x)][self.stateIndex_ls.index(y)]+=1
        for (i, r) in enumerate(self.transitionMatrix_ar):
            if np.sum(self.transitionMatrix_ar[i])>0:
                self.transitionMatrix_ar[i]=self.transitionMatrix_ar[i]/sum(self.transitionMatrix_ar[i])
                
        # Take the log of the transition matrix to make
        # calculations more accurate
        self.logTransitionMatrix_ar=np.log(self.transitionMatrix_ar)
        
        # Same for the initial states:
        self.logInitialState_ar=np.log(self.initialState_ar)
        
    def apply(self, stateIn_ms, steps=1):
        '''
        Takes an input MarkovState, and returns the output
        MarkovState following steps applications.
        
        Can also take a list of states as an alternative to
        the input MarkovState, in which case it will be 
        converted as necessary.

        Both stateIn_ar and transitionIn_ar are expressed as logs.
        
        TODO: Raise an error if indices don't match, or if a
              list is input which contains nonexistent states.
        '''
        
        if not isinstance(stateIn_ms, MarkovState):
            # Need to raise an error if it's not list-like,
            # if it's empty, or if there's a state that's
            # not in the transition matrix
            #
            # For non-list-like, read "doesn't support count()"
            initialStatesDist_ls=[stateIn_ms.count(s) for s in self.stateIndex_ls]
            dist_ar=np.array([initialStatesDist_ls])/np.sum(initialStatesDist_ls)
            stateIn_ms=MarkovState(self.stateIndex_ls,
                                   np.log(dist_ar),
                                   [])
        return stateIn_ms


    if False:
        stateOut_ar=np.empty(stateIn_ar.shape)
        transOut_ar=np.zeros(stateIn_ar.shape, dtype=np.int)

        for (i, x) in enumerate(stateIn_ar):
            calculateTransitions_ar=stateIn_ar + transitionMatrix_ar[i]
            argmax_i=np.argmax(calculateTransitions_ar)

            stateOut_ar[i]=calculateTransitions_ar[argmax_i]
            transOut_ar[i]=argmax_i


        return (stateOut_ar, transOut_ar)
    


Now, as well as the Markov model class, I also want a MarkovState class. The MarkovState class will contain the information about the state following one or more transitional steps from the MM.

This class needs the following data:

1. An index (index_ls) of the states considered in the MarkovState

2. A (logged) probabilistic distribution of the current state (currentState_ar)

3. A structure containing the historical paths to each node in the index.

In [45]:
class MarkovState:
    
    def __init__(self, index_ls, currentState_ar, paths_ls):
        
        self.myIndex_ls=index_ls
        self.myCurrentState_ar=currentState_ar
        self.myPaths_ls=paths_ls
        
        
        
    

In [46]:
u=MarkovState(1, 2, 3)

In [47]:
isinstance(u, MarkovState)

True

In [48]:
transitions_ls=[('a', 'a'), ('a', 'b'), ('a', 'b'), ('a', 'b'), ('a', 'b'), 
                ('a', 'c'), ('a', 'c'), ('a', 'c'), ('a', 'c'), ('a', 'c'), 
                ('b', 'a'), ('b', 'b'), ('b', 'b'), ('b', 'b'), ('b', 'b'), 
                ('b', 'b'), ('b', 'b'), ('b', 'b'), ('b', 'b'), ('b', 'c'),
                ('c', 'a'), ('c', 'a'), ('c', 'a'), ('c', 'b'), ('c', 'b'),
                ('c', 'b'), ('c', 'b'), ('c', 'c'), ('c', 'c'), ('c', 'c')]

In [49]:
m=MarkovModel(transitions_ls)

In [50]:
u=m.apply(['a', 'b', 'b', 'd'])

/Users/alistairwillis/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:70: RuntimeWarning: divide by zero encountered in log


In [53]:
u.myCurrentState_ar

array([[-1.09861229, -0.40546511,        -inf]])